In [1]:
import json
import math
from functools import partial
from pathlib import Path
from configparser import ConfigParser

import umap
import hdbscan
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from plotly.graph_objs import Scatter, Layout, Heatmap
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import chart_studio
import chart_studio.plotly as py

from src.aggregator import *

pd.set_option('display.max_colwidth', -1)

C:\Users\RistoHinno\AppData\Local\Temp\ipykernel_12332\3496233029.py:22: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


## Data

In [2]:
df=pd.read_csv('data/interim/stenos_cleaned_topics.csv')
df.shape

(209949, 22)

In [3]:
#normalize factions
df['fation']=df.fation.replace({'Fraktsiooni mittekuuluvad saadikud': 'Fraktsioonidesse mittekuulunud Riigikogu liikmed',
                               'Fraktsiooni mittekuuluvad Riigikogu liikmed':'Fraktsioonidesse mittekuulunud Riigikogu liikmed'})

In [4]:
df_topic_clusters=pd.read_excel('data/interim/riigikogu_topics_clusters.xlsx')
df_topic_clusters['topic_words3']=df_topic_clusters.words.apply(lambda x: ' '.join(x.split()[:3]))
df_topic_clusters.shape

(1152, 9)

## Colors for plotting

In [5]:
colors=json.loads(Path('utils/colors.json').read_text())
colors=list(colors.keys())[10:]

## Add topics to original data

In [6]:
df=pd.merge(df, df_topic_clusters[['topic_cluster_id','topic_id', 'x_rand', 'y_rand']], 
            left_on='topic_id', right_on='topic_id', how='left')
df.shape

(209949, 25)

## Clusters which are noninformative

In [7]:
clusters_noninform=pd.read_csv('data/interim/noninform_clusters.csv')['0'].tolist()
topics_noninform=pd.read_csv('data/interim/noninform_topics.csv')['0'].tolist()

## Topics related to environment

In [15]:
topics_environment=[['energiaagentuur', 'agentuuri', 'agentuur'],
                   ['korterelamute', 'kortermajade', 'kredexi'],
                   ['metsa', 'metsade', 'mets'],
                   ['elektri', 'elektrit', 'taastuvenergia'],
                   ['utah', 'jordaania', 'energia'],
                   ['loomi', 'loomade', 'loomad'],
                   ['muundatud', 'geneetiliselt', 'gmo'],
                   ['hoone', 'rkas', 'hoonet'],
                   ['oiglase', 'virumaale', 'ulemineku'],
                   ['kala', 'kalurid', 'kalurite'],
                   ['jaatmeid', 'liigiti', 'prugi'],
                   ['planeeringu', 'keskkonnamoju', 'keskkonnamojude'],
                   ['kala', 'kalurid', 'kalurite'],
                   ['lng', 'terminali', 'gaasi'],
                   ['est', 'kavatsuste', 'for']]

topics_env_str=[' '.join(lst) for lst in topics_environment]

In [16]:
df['topic_env']=np.where(df.topic_words3.isin(topics_env_str), df.topic_words3, 'other')

In [17]:
df['topic_env'].value_counts()

other                                        204479
elektri elektrit taastuvenergia              1748  
metsa metsade mets                           757   
hoone rkas hoonet                            492   
loomi loomade loomad                         385   
korterelamute kortermajade kredexi           357   
lng terminali gaasi                          346   
jaatmeid liigiti prugi                       333   
kala kalurid kalurite                        245   
oiglase virumaale ulemineku                  228   
planeeringu keskkonnamoju keskkonnamojude    172   
utah jordaania energia                       167   
energiaagentuur agentuuri agentuur           119   
muundatud geneetiliselt gmo                  80    
est kavatsuste for                           41    
Name: topic_env, dtype: int64

In [14]:
df_topic_clusters

,topic_id,num_docs,words,x_rand,y_rand,topic_size,topic_cluster_id,color_topic_cluster,topic_words3
0,0,2836,koolis kooli opilased kool koolides opilaste opilasi koolid koolide pohikoolis,0.832609,12.465334,2836,18,darkslateblue,koolis kooli opilased
1,1,2226,oelge te saite teilt hakkasite teie olete vastake raakige tegite,1.093928,9.741197,2226,3,brown,oelge te saite
2,2,2047,komisjonis teemat kasitlenud arutatud arutanud pohjalikult arutasime teemasid temaatikat arutati,1.444144,10.505503,2047,8,darkgray,komisjonis teemat kasitlenud
3,3,2015,menetluslikud konsensuslikult konsensuslikud lugemisele menetlusotsused lugemiseks konsensuslik paevakorda novembriks taiskogule,2.818501,10.239195,2015,8,darkgray,menetluslikud konsensuslikult konsensuslikud
4,4,1963,maaran loppenud lopp lopetatud istung sulgen kell konesoove ammendatud avata,2.291581,8.136870,1963,1,blue,maaran loppenud lopp
...,...,...,...,...,...,...,...,...,...
1147,1147,18,laurson peeter kaevandamise laasi valeri tulik ernits vosa toobali ilmneb,7.801987,6.899067,18,0,black,laurson peeter kaevandamise
1148,1148,18,sibul priit toobal sibula aafrika hepneri kesk seakatku espenberg jufereva,4.804576,-0.205021,18,0,black,sibul priit toobal
1149,1149,18,einar aegi liivamagi olga vladimir tramberg paur lokk raivo laasi,8.263407,8.190639,18,9,darkgreen,einar aegi liivamagi
1150,1150,17,aivar kokk rene soerd koka merry aart soerdi algatajale juunist,10.873006,10.670400,17,5,darkblue,aivar kokk rene
